In [74]:
import random
import math
import matplotlib.pyplot as plt

In [ ]:
def argmaxall(gen):
    maxv = -math.inf
    maxvals = []
    for (e,v) in gen:
        if v>maxv:
            maxvals,maxv = [e], v
        elif v==maxv:
            maxvals.append(e)
    return maxvals

In [76]:
def argmaxe(gen):
    return random.choice(argmaxall(gen))


In [77]:
def argmax(lst):
    return argmaxe(enumerate(lst))

In [78]:
def flip(prob):
    return random.random() < prob

In [79]:
def select_from_dist(item_prob_dist):
    ranreal = random.random()
    for (it,prob) in item_prob_dist.items():
        if ranreal < prob:
            return it
        else:
            ranreal -= prob
    raise RuntimeError(f"{item_prob_dist} is not a probability distribution")

In [80]:
class Displayable:
    max_dispaly_level = 1
    def display(self, level=0, *args, **nargs):
        if level <= self.max_dispaly_level:
            self.__display(level, *args, **nargs)

In [81]:
class Plot_history(object):
    def __init__(self, ag, env):
        self.ag = ag
        self.env = env
        plt.ion()
        plt.xlabel('Time')
        plt.ylabel('Value')

    def plot_env_hist(self):
        num = len(env.stock_history)
        plt.plot(range(num),env.price_history,label="Price")
        plt.plot(range(num),env.stock_history,label="In stock")
        plt.legend()

    def plot_agent_hist(self):
        """plot history of buying"""
        num = len(ag.buy_history)
        plt.bar(range(1,num+1), ag.buy_history, label="Bought")
        plt.legend()




In [82]:
class Agent(Displayable):

    def initial_action(self, percept):
        return self.select_action(percept)
    def select_action(self, percept):
        raise NotImplementedError("go")

In [83]:

class Environment(Displayable):
    def initial_percept(self):
        raise NotImplementedError("initial_percept")

    def do(self, action):
        raise NotImplementedError("Environment.do")

In [84]:
class Simulate(Displayable):

    def __init__(self, agent, environment):
        self.agent = agent
        self.env = environment
        self.percept = self.env.initial_percept()
        self.percept_history = [self.percept]
        self.action_history = []

    def go(self, n):
        for i in range(n):
            action = self.agent.select_action(self.percept)
            print(f"i={i} action={action}")

            self.percept = self.env.do(action, i)
            print(f"      percept={self.percept}")


In [85]:
class TP_env(Environment):
    price_delta = [0, 0, 0, 21, 0, 20, 0, -64, 0, 0, 23, 0, 0, 0, -35,
        0, 76, 0, -41, 0, 0, 0, 21, 0, 5, 0, 5, 0, 0, 0, 5, 0, -15, 0, 5,
       0, 5, 0, -115, 0, 115, 0, 5, 0, -15, 0, 5, 0, 5, 0, 0, 0, 5, 0,
       -59, 0, 44, 0, 5, 0, 5, 0, 0, 0, 5, 0, -65, 50, 0, 5, 0, 5, 0, 0,
       0, 5, 0]
    sd = 5

    def __init__(self):
        self.time=0
        self.stock=20
        self.stock_history = []
        self.price_history = []

    def initial_percept(self):
        self.stock_history.append(self.stock)
        self.price = round(234+self.sd*random.gauss(0,1))
        self.price_history.append(self.price)
        return {'price': self.price,
                'instock': self.stock}

    def do(self, action, time_unit):
        used = select_from_dist({6:0.1, 5:0.1, 4:0.1, 3:0.3, 2:0.2, 1:0.2})
        print(f"i={time_unit} used={used}")
        bought = action['buy']
        self.stock = self.stock+bought-used
        self.stock_history.append(self.stock)
        self.time += 1
        self.price =  round(self.price
                        + self.price_delta[self.time%len(self.price_delta)]
                        + self.sd*random.gauss(0,1))
        self.price_history.append(self.price)
        return {'price': self.price,
                'instock': self.stock}

In [86]:
class PriceMonitoringController:
    def __init__(self, threshold_discount=0.2):
        self.threshold_discount = threshold_discount

    def is_price_below_threshold(self, current_price, average_price):
        return current_price < (1 - self.threshold_discount) * average_price

In [87]:
class InventoryMonitoringController:
    def __init__(self, critical_threshold=10):
        self.critical_threshold = critical_threshold

    def is_inventory_critical(self, current_stock):
        return current_stock < self.critical_threshold

In [88]:
class OrderingController:
    def __init__(self):
        self.buy_history = []
        self.spent = 0

    def decide_order(self, current_price, average_price, current_stock, price_controller, inventory_controller):
        if price_controller.is_price_below_threshold(current_price, average_price) and current_stock >= 10:
            tobuy = 15
        elif inventory_controller.is_inventory_critical(current_stock):
            tobuy = 10
        else:
            tobuy = 0

        self.buy_history.append(tobuy)
        self.spent += tobuy * current_price
        return tobuy

In [89]:
class TP_agent(Agent):
    def __init__(self):
        percept = env.initial_percept()
        self.ave = self.last_price = percept['price']
        self.instock = percept['instock']

        # Initialize controllers
        self.price_controller = PriceMonitoringController(threshold_discount=0.2)
        self.inventory_controller = InventoryMonitoringController(critical_threshold=10)
        self.ordering_controller = OrderingController()

    def select_action(self, percept):
        self.last_price = percept['price']
        self.ave = self.ave + (self.last_price - self.ave) * 0.05
        self.instock = percept['instock']

        tobuy = self.ordering_controller.decide_order(
            self.last_price,
            self.ave,
            self.instock,
            self.price_controller,
            self.inventory_controller
        )
        return {'buy': tobuy}


In [90]:

env = TP_env()
ag = TP_agent()
sim = Simulate(ag,env)
sim.go(10)
ag.ordering_controller.spent/env.time

i=0 action={'buy': 0}
i=0 used=2
      percept={'price': 234, 'instock': 18}
i=1 action={'buy': 0}
i=1 used=2
      percept={'price': 239, 'instock': 16}
i=2 action={'buy': 0}
i=2 used=5
      percept={'price': 253, 'instock': 11}
i=3 action={'buy': 0}
i=3 used=3
      percept={'price': 255, 'instock': 8}
i=4 action={'buy': 10}
i=4 used=4
      percept={'price': 273, 'instock': 14}
i=5 action={'buy': 0}
i=5 used=4
      percept={'price': 270, 'instock': 10}
i=6 action={'buy': 0}
i=6 used=3
      percept={'price': 211, 'instock': 7}
i=7 action={'buy': 10}
i=7 used=1
      percept={'price': 217, 'instock': 16}
i=8 action={'buy': 0}
i=8 used=3
      percept={'price': 218, 'instock': 13}
i=9 action={'buy': 0}
i=9 used=3
      percept={'price': 241, 'instock': 10}


466.0

In [91]:
sim.go(100);



i=0 action={'buy': 0}
i=0 used=1
      percept={'price': 236, 'instock': 9}
i=1 action={'buy': 10}
i=1 used=3
      percept={'price': 237, 'instock': 16}
i=2 action={'buy': 0}
i=2 used=2
      percept={'price': 230, 'instock': 14}
i=3 action={'buy': 0}
i=3 used=2
      percept={'price': 197, 'instock': 12}
i=4 action={'buy': 0}
i=4 used=3
      percept={'price': 195, 'instock': 9}
i=5 action={'buy': 10}
i=5 used=2
      percept={'price': 279, 'instock': 17}
i=6 action={'buy': 0}
i=6 used=2
      percept={'price': 273, 'instock': 15}
i=7 action={'buy': 0}
i=7 used=6
      percept={'price': 232, 'instock': 9}
i=8 action={'buy': 10}
i=8 used=3
      percept={'price': 231, 'instock': 16}
i=9 action={'buy': 0}
i=9 used=3
      percept={'price': 234, 'instock': 13}
i=10 action={'buy': 0}
i=10 used=3
      percept={'price': 239, 'instock': 10}
i=11 action={'buy': 0}
i=11 used=4
      percept={'price': 263, 'instock': 6}
i=12 action={'buy': 10}
i=12 used=3
      percept={'price': 262, 'instock